In [47]:
import pandas as pd
import numpy as np
import math
from pulp import *
from tabulate import tabulate

In [181]:
# ========= SET-UP =========== #
NUM_AREA = 2
R = 11.5 # Revenue
S = 1.5 # Multiplier

# ========== DATA ============= #
x_ij = [[0.985, 0.615], [0.615, 0.985]] # [[row]] - x00, x01, x10, x11
c_ij = [[0, 2], [2, 0]] # c00, c01, c10, c11

d_i = [60, 20] # d_1, d_2
n_0i = [10, 200] # n_01, n_02

# ========= INIT LP ============ #
prob = LpProblem("Model", LpMaximize)

m_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
y_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
n_1i = []
for i in range(NUM_AREA):
    n_1i.append(LpVariable("n_1%d"%(i), 0, None, LpInteger))
    for j in range(NUM_AREA):
        y_ij[i][j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)
        if i!=j:
            m_ij[i][j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

# ========= CONSTRUCT LP ============ #

# Objective
prob += sum([S*R*y_ij[i][j]*x_ij[i][j] - m_ij[i][j]*c_ij[i][j] for i in range(NUM_AREA) for j in range(NUM_AREA)])

# Subject To
for j in range(NUM_AREA):
    # n_1i
    prob += n_1i[j] == n_0i[j] + sum(m_ij[i][j] for i in range(NUM_AREA)) - sum(m_ij[j][i] for i in range(NUM_AREA)), "Equation Area: %d"%j
    # n_0i
    prob += n_0i[j] >= sum(m_ij[j][i] for i in range(NUM_AREA)), "Moving Constrain: %d"%j
    # y_ij
    prob += sum(y_ij[j][i] for i in range(NUM_AREA)) <= d_i[j] ,"Total Demand: %d"%j
    prob += sum(y_ij[i][j]*x_ij[i][j] for i in range(NUM_AREA)) <= n_1i[j] ,"Total Available: %d"%j
    

# ============== SOLVE ============= #
prob.solve()
print("\n ======== Optimal Variables ========= \n")
for v in prob.variables():
    print (v.name, "=", v.varValue)
print("Objective: " ,value(prob.objective))

print("\n ======== Model Details ========= \n")
results = []
for name, c in list(prob.constraints.items()):
    results.append([name,c,  c.pi,  c.slack])
print (tabulate(results, headers=['Constrain', 'Formula', "pi", "slack"]))


 ======== Optimal Variables ========= 

m_01 = 0.0
m_10 = 50.0
n_10 = 60.0
n_11 = 150.0
y_00 = 60.0
y_01 = 0.0
y_10 = 0.0
y_11 = 20.0
Objective:  1259.3000000000002

 ======== Model Details ========= 

Constrain            Formula                                    pi    slack
-------------------  ---------------------------------------  ----  -------
Equation_Area:_0     m_01 - m_10 + n_10 = 10                    -0     -0
Moving_Constrain:_0  m_01 <= 10                                 -0     10
Total_Demand:_0      y_00 + y_01 <= 60                          -0     -0
Total_Available:_0   -n_10 + 0.985*y_00 + 0.615*y_10 <= -0.0    -0      0.9
Equation_Area:_1     -m_01 + m_10 + n_11 = 200                  -0     -0
Moving_Constrain:_1  m_10 <= 200                                -0    150
Total_Demand:_1      y_10 + y_11 <= 20                          -0     -0
Total_Available:_1   -n_11 + 0.615*y_01 + 0.985*y_11 <= -0.0    -0    130.3
